Idea of this analysis to deepdive into C trips. C trips are those which were not dispatched by the central or called specifically for any driver. These are trips that originated randomly, at any hour of the day from anywhere.

Would be interesting to compare heatmaps for a weekday (say Monday) to a weekend (say Friday or even Sunday).

Also can compare a day just prior to a public holiday or on a public holiday V/S anyother random day.

In [1]:
import pickle

routes = pickle.load( open( "save.p", "rb" ) )

In [2]:
import folium
from folium import plugins
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
%%time
data = routes[routes.MISSING_DATA == False]
data = data[data.POLYLINE != '[]']
data = data[['TRIP_ID', 'CALL_TYPE', 'POLYLINE', 'TIMESTAMP']]

data['PICK_UP_LOCATION'] = data.POLYLINE.apply(lambda x: eval(x.split()[0])[0])
data['PICK_UP_LOCATION'] = data.PICK_UP_LOCATION.apply(lambda x: np.flip(x,0))
data['DISTANCE'] = data.POLYLINE.apply(lambda x: len(x))
data['TIME_TRIP_MIN'] = data.DISTANCE.apply(lambda x: float((x-1)*15)/60)
data['WEEK_DAY'] = data.TIMESTAMP.apply(lambda x: dt.datetime.fromtimestamp(x).weekday())
data = data.loc[:, data.columns != 'index']
data.head()

Wall time: 17min 49s


,TRIP_ID,CALL_TYPE,POLYLINE,TIMESTAMP,PICK_UP_LOCATION,DISTANCE,TIME_TRIP_MIN,WEEK_DAY
0,1372636858620000589,C,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",1372636858,"[41.141412, -8.618643]",500,124.75,6
1,1372637303620000596,B,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",1372637303,"[41.159826, -8.639847]",413,103.00,6
2,1372636951620000320,C,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",1372636951,"[41.140359, -8.612964]",1408,351.75,6
3,1372636854620000520,C,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",1372636854,"[41.151951, -8.574678]",938,234.25,6
4,1372637091620000337,C,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",1372637091,"[41.18049, -8.645994]",634,158.25,6


In [4]:
import pickle
pickle.dump(data, open( "heat.p", "wb" ) )

In [5]:
#RUN FROM HERE IF ALREADY PICKLED
import pickle
data = pickle.load( open( "heat.p", "rb" ) )

In [6]:
polyline_source = data.copy()
polyline_source = polyline_source[polyline_source.CALL_TYPE == 'C']
mondays_pick_up = polyline_source[polyline_source.WEEK_DAY == 0].PICK_UP_LOCATION.tolist()
sundays_pick_up = polyline_source[polyline_source.WEEK_DAY == 6].PICK_UP_LOCATION.tolist()
list_mondays_pick_up = []
list_sundays_pick_up = []
#list_a_pick_up = []
for i in mondays_pick_up:
    list_mondays_pick_up.append(i.tolist())
    
for i in sundays_pick_up:
    list_sundays_pick_up.append(i.tolist())

In [7]:
# Create Monday Map
monday = folium.Map([41.155, -8.63], zoom_start=9)
# Add heatMap 
plugins.HeatMap(list_mondays_pick_up, radius=11).add_to(monday)
# Print heatMap
monday

In [8]:
# Create Sunday Map
sunday = folium.Map([41.155, -8.63], zoom_start=9)
# Add heatMap 
plugins.HeatMap(list_sundays_pick_up, radius=11).add_to(sunday)
# Print heatMap
sunday